
# QLoRA Fine-tuning de **Qwen2.5-7B-Instruct** en Google Colab T4 (SAMSum, Resumen de Diálogos)

se realiza un **fine-tuning supervisado (SFT)** usando **QLoRA (4-bit)** sobre el modelo preentrenado **Qwen2.5-7B-Instruct** para la tarea de **resumen de diálogo** con el dataset **[SAMSum](https://huggingface.co/datasets/samsum)**. 

Ejecución: **Google Colab con GPU T4 (16 GB)**.

**pasos:**
- Instalación de librerías (`transformers`, `datasets`, `peft`, `bitsandbytes`, `trl`, `evaluate`, `rouge_score`).
- Carga del **modelo Qwen2.5-7B-Instruct** en 4-bit.
- **Plantillas de prompt** usando `tokenizer.apply_chat_template`.
- **Entrenamiento con SFTTrainer** (TRL) + **LoRA** (PEFT).
- **Inferencia** y **evaluación rápida** con ROUGE en una muestra pequeña.
- Guardado de los **adaptadores LoRA**.

> Opción: **QA generativo** (SQuAD v1.1) u otra tarea, ver la sección “Variantes” al final.


## 1) Instalación y verificación de entorno

In [ ]:

!pip -q install "transformers>=4.43.0" "datasets>=2.19.0" "accelerate>=0.32.0" "bitsandbytes>=0.43.0"                  "peft>=0.11.0" "trl>=0.9.4" "evaluate>=0.4.2" "rouge-score>=0.1.2"


In [ ]:
!pip install -U bitsandbytes trl

In [ ]:
from huggingface_hub import login
# tu token
login("hf_xxxxxxxxxxxxx")

In [ ]:
import torch, os, platform
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("Ejecuta este cuaderno en Colab con GPU (T4 recomendada).")


## 2) Configuración del modelo y cuantización 4-bit (QLoRA)

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"   # Alternativa: "Qwen/Qwen2.5-7B"
USE_BF16 = True                         # Si tu GPU soporta bfloat16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Recomendado para entrenamiento estable con checkpointing
model.config.use_cache = False
model.gradient_checkpointing_enable()


## 3) Cargar dataset: SAMSum (diálogos y resúmenes)

In [ ]:

from datasets import load_dataset
ds = load_dataset("knkarthick/samsum")
print(ds)

# Para Colab T4, usar subconjuntos pequeños para una demo rápida
train_samples = 3000   # ajusta según tu tiempo/VRAM
eval_samples  = 300    # idem

train_ds = ds["train"].shuffle(seed=42).select(range(min(train_samples, len(ds["train"]))))
eval_ds  = ds["validation"].shuffle(seed=42).select(range(min(eval_samples, len(ds["validation"]))))
print(train_ds, eval_ds)


## 4) Plantilla de *prompt* con `apply_chat_template`

In [ ]:

def format_chat(dialogue: str, summary: str = None, for_train: bool = True) -> str:
    """
    Crea un prompt tipo chat para Qwen2.5.
    - for_train=True: incluye la respuesta (summary) como mensaje del assistant.
    - for_train=False: agrega el "generation prompt" para inferencia.
    """
    """
    messages = [
        {"role": "system", "content": "Eres un asistente que resume diálogos en una sola oración, breve y fiel al contenido."},
        {"role": "user", "content": f"Resume el siguiente diálogo:\n\n{dialogue.strip()}"},
    ]
    """

    messages = [
        {"role": "system", "content": "You are an assistant that summarizes dialogues into a single sentence, short and faithful to the content."},
        {"role": "user", "content": f"Summarize the following dialogue:\n\n{dialogue.strip()}"},
    ]

    if for_train and summary is not None:
        messages.append({"role": "assistant", "content": summary.strip()})
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    else:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Ejemplo:
print(format_chat(train_ds[0]["dialogue"], train_ds[0]["summary"])[:500])


## 5) Preprocesamiento: convertir a campo `text`

In [ ]:

def to_sft_format(example, for_train=True):
    text = format_chat(example["dialogue"], example.get("summary"), for_train=for_train)
    return {"text": text}

train_sft = train_ds.map(lambda ex: to_sft_format(ex, True), remove_columns=train_ds.column_names)
eval_sft  = eval_ds.map(lambda ex: to_sft_format(ex, True), remove_columns=eval_ds.column_names)

print(train_sft[0]["text"][:300])


## 6) Entrenamiento con TRL `SFTTrainer` + PEFT (LoRA)

In [ ]:
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)

train_args = SFTConfig(
    output_dir="qwen25-7b-samsum-lora",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="steps",
    eval_steps=100,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    bf16=True,
    optim="paged_adamw_8bit",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_sft,
    eval_dataset=eval_sft,
    peft_config=peft_config,
    args=train_args,
    packing=False,
)

trainer.train()
trainer.model.save_pretrained("qwen25-7b-samsum-lora")
tokenizer.save_pretrained("qwen25-7b-samsum-lora")

## 7) Inferencia en ejemplos de validación

In [ ]:

import textwrap, torch

def generate_summary(dialogue: str, max_new_tokens=128, temperature=0.7, top_p=0.9):
    prompt = format_chat(dialogue, None, for_train=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
        )
    gen = tokenizer.decode(out[0], skip_special_tokens=True)
    return gen.split("assistant")[-1].strip()

ex = eval_ds[0]
pred = generate_summary(ex["dialogue"])
print("=== DIÁLOGO ===")
print(textwrap.shorten(ex["dialogue"], width=1200))
print("\n=== RESUMEN GOLD ===\n", ex["summary"])
print("\n=== RESUMEN GENERADO ===\n", pred)


## 8) Evaluación rápida (ROUGE) en una muestra

In [ ]:

import evaluate
rouge = evaluate.load("rouge")

n_eval = 50
subset = eval_ds.select(range(min(n_eval, len(eval_ds))))

preds, refs = [], []
for ex in subset:
    preds.append(generate_summary(ex["dialogue"], max_new_tokens=96))
    refs.append(ex["summary"])

scores = rouge.compute(predictions=preds, references=refs, use_aggregator=True)
print(scores)



## 9) Opciones
- **QA Generativo (SQuAD v1.1):**
  - `load_dataset("squad")` y define el prompt con contexto + pregunta + respuesta.
- **Extracción estructurada (JSON):**
  - Cambia el `system` a “Devuelve **solo** JSON válido …” y usa cadenas JSON en `assistant`.
- **Instrucción general (Dolly-15k / OASST):**
  - Sustituye SAMSum por un instruction dataset.
- **Optimización T4:**
  - Baja `max_seq_length`, sube `gradient_accumulation_steps`, o reduce `train_samples`.
